In [3]:
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import numpy as np

### Test MTCNN face detection model

In [4]:

class MTCNNFaceNetDetector:
    def __init__(self, device='cpu'):
        self.device = device
        
        # Face detector
        self.mtcnn = MTCNN(
            image_size=160, 
            margin=0, 
            min_face_size=20,
            device=device,
            select_largest=True  # Select largest face if multiple
        )
        
        # Face recognition model (embeddings)
        self.resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
    
    def get_embedding(self, image_path):
        """Extract face embedding from image"""
        img = Image.open(image_path).convert('RGB')
        
        # Detect and crop face
        face = self.mtcnn(img)
        
        if face is None:
            return None
        
        # Get embedding
        with torch.no_grad():
            face = face.unsqueeze(0).to(self.device)
            embedding = self.resnet(face).cpu().numpy()[0]
        
        return embedding
    
    def compare_faces(self, embedding1, embedding2):
        """Compare two embeddings, return distance"""
        if embedding1 is None or embedding2 is None:
            return None
        
        # Euclidean distance
        distance = np.linalg.norm(embedding1 - embedding2)
        return distance

In [ ]:
image_path = "/workspaces/face_duplicate_detection/data/images/Aaron Eckhart_1.jpg"


In [ ]:
# Usage
detector = MTCNNFaceNetDetector()

emb1 = detector.get_embedding('person1_photo1.jpg')
emb2 = detector.get_embedding('person1_photo2.jpg')

if emb1 is not None and emb2 is not None:
    distance = detector.compare_faces(emb1, emb2)
    print(f"Distance: {distance:.3f}")
    # Typical threshold: < 0.7 for same person